In [ ]:
from data_extraction import DataExtractor
import pandas as pd
import numpy as np
import boto3
from io import StringIO
import re



In [ ]:
product_df = DataExtractor.extract_from_s3()
pd.set_option('display.max_rows', None)

In [ ]:
product_df.info()


In [ ]:
product_df

In [ ]:
import data_cleaning 

product_df = DataCleaning.convert_product_weights(product_df)

In [ ]:
class DataCleaning:
    @staticmethod
    def convert_product_weights(products_df):
        """Converts product weights to kg and cleans the weight column."""

        def convert_to_kg(value):
            """Helper function to convert weight to kg."""
            # If the value is already a float (indicating no units), return it as is
            if isinstance(value, float):
                return value

            try:
                # Detect calculation (e.g., "2x250ml") and evaluate it
                if 'x' in value:
                    parts = value.split('x')
                    if len(parts) == 2:
                        num, unit_value = parts
                        # Remove non-numeric characters from unit_value for multiplication
                        unit_value = re.sub(r'[^0-9.]', '', unit_value)
                        # Perform the multiplication and then continue with the conversion
                        value = str(float(num) * float(unit_value))


                # Check for 'kg' in weight and convert to float
                if 'kg' in value:
                    return float(value.replace('kg', ''))
                elif '77g .' == value:  # Specific case with an extra period
                    return 77.0 / 1000  # Convert 77g to kg
                # Check for 'g' in weight and convert to kg
                elif 'g' in value:
                    return float(value.replace('g', '')) / 1000
                # Check for 'ml' and assume 1:1 ratio with 'g', then convert to kg
                elif 'ml' in value:
                    return float(value.replace('ml', '')) / 1000
                elif value == '16oz':  # Specific case with ounces
                    return 16 * 0.0283495  # Convert 16oz to kg
                else:
                    # If no known unit, assume it's already in kg
                    return float(value)
            except ValueError as e:
                # If there's a ValueError, print it and return None to handle it later
                print(f"Error converting '{value}': {e}")
                return None

        # Apply the conversion to each entry in the weight column
        products_df['weight'] = products_df['weight'].apply(convert_to_kg)
        
        # If there was an error during conversion, the result will be None, so we can drop these rows or handle them
        products_df = products_df.dropna(subset=['weight'])
        
        return products_df

# Instantiate the DataCleaning class
data_cleaner = DataCleaning()

# Apply the method to the DataFrame
df_cleaned_weights = data_cleaner.convert_product_weights(product_df)

In [ ]:
class DataCleaning:
    @staticmethod
    def convert_to_kg(value):
        """Helper function to convert weight to kg."""
        # If the value is already a float (indicating no units), return it as is
        if isinstance(value, float):
            return value
        
        try:
            # Detect calculation (e.g., "2x250ml") and evaluate it
            if 'x' in value:
                parts = value.split('x')
                if len(parts) == 2:
                    num, unit_value = parts
                    # Remove non-numeric characters from unit_value for multiplication
                    unit_value = re.sub(r'[^0-9.]', '', unit_value)
                    # Perform the multiplication and then continue with the conversion
                    value = str(float(num) * float(unit_value))

            # Check for 'kg' in weight and convert to float
            if 'kg' in value:
                return float(value.replace('kg', ''))
            # Check for 'g' in weight and convert to kg
            elif 'g' in value:
                return float(value.replace('g', '')) / 1000
            # Check for 'ml' and assume 1:1 ratio with 'g', then convert to kg
            elif 'ml' in value:
                return float(value.replace('ml', '')) / 1000
            # Check for specific cases that caused errors before
            elif value == '77g .':  # Specific case with an extra period
                return 77.0 / 1000  # Convert 77g to kg
            elif value == '16oz':  # Specific case with ounces
                return 16 * 0.0283495  # Convert 16oz to kg
            else:
                # If no known unit, assume it's already in kg
                return float(value)
        except ValueError as e:
            # If there's a ValueError, print it and return None to handle it later
            print(f"Error converting '{value}': {e}")
            return None


    @staticmethod       
    def convert_product_weights(self, products_df):
        """Converts product weights to kg and cleans the weight column."""
        # Apply the conversion to each entry in the weight column
        products_df['weight'] = products_df['weight'].DataCleaning.convert_to_kg()
        
        # If there was an error during conversion, the result will be None, so we can drop these rows or handle them
        #products_df = products_df.dropna(subset=['weight'])
        
        return products_df

# Instantiate the DataCleaning class
data_cleaner = DataCleaning()

# Apply the method to the DataFrame
df_cleaned_weights = data_cleaner.convert_product_weights(product_df['weight'])


In [ ]:
import re 

print(product_df['weight'].sample(50))

conversion_factors = {"g" : 1, "mg" :0.001, "ml": 1, 'kg': 1000}

def convert_weight(weight):
    if isinstance(weight, str):
        match = re.match(r'(\d+\.?\d*)\s*([a-zA-Z]+)', weight)
        if match:
            value, unit = match.groups()
            value = float(value)
            unit = unit.lower()
            #Convert the values to grams        
            value *= conversion_factors.get(unit, 1)
            #Divide by 1000 to convert to kilograms
            return value/1000
        else:
            print(f'No unit found for weight: {weight}')
            return weight
        
product_df['weight_kg'] = product_df['weight'].apply(convert_weight)

In [ ]:
import re 

product_df = DataExtractor.extract_from_s3()

#print(product_df['weight'].sample(50))

conversion_factors = {"g" : 1, "mg" :0.001, "ml": 1, 'kg': 1000}

def convert_weight(weight):
    if isinstance(weight, str):
        match = re.match(r'(\d+\.?\d*)\s*([a-zA-Z]+)', weight)
        if match:
            value, unit = match.groups()
            value = float(value)
            unit = unit.lower()
            #Convert the values to grams        
            value *= conversion_factors.get(unit, 1)
            #Divide by 1000 to convert to kilograms
            return value/1000
        else:
            print(f'No unit found for weight: {weight}')
            return weight
    elif isinstance(weight, (int, float)):
        return weight/1000
    
product_df['weight_kg'] = product_df['weight'].apply(convert_weight)
print(product_df['weight_kg'].unique())

In [ ]:
def convert_weight(weight):
    # Match patterns like '12 x 100g' or '6 x 400ml' 
    if isinstance(weight, str):
        match = re.match(r'(\d+)\s*x\s*(\d+\.?\d*)([a-zA-Z]+)', weight)
        if match:
            quantity, value, unit = match.groups()
            quantity = float(quantity)
            value = float(value)
            unit = unit.lower()
            #Convert the values to grams
            if unit in conversion_factors:
                value *= conversion_factors[unit]
            #Divid by 1000 since we want everything in kg
            return (quantity * value) / 1000
        else:
            print(f'No unit found for weight: {weight}')
            return None
    elif isinstance(weight, (int, float)):
            return weight/1000
    
product_df['weight_kg'] = product_df['weight'].apply(convert_weight)
print(product_df['weight_kg'])


In [ ]:
store_data = DataExtractor.retrieve_stores_data()

In [ ]:
print(product_df['weight'].unique())
print(product_df['weight'][1852])

In [ ]:
store_data.replace('NULL', np.nan, inplace=True)
store_data = store_data.dropna(subset=['store_type'])
valid_store_types = ['Local', 'Super Store', 'Mall Kiosk', 'Outlet']
store_data = store_data[store_data['store_type'].isin(valid_store_types)]
store_data['opening_date'] = pd.to_datetime(store_data['opening_date'], errors = 'coerce', format ='%Y-%m-%d')
store_data = store_data.drop_duplicates(subset=['address'])
store_data['staff_numbers'] = pd.to_numeric(store_data['staff_numbers'], errors = 'coerce')
#store_data['staff_numbers'] = store_data['staff_numbers'].astype(int)
valid_country_codes = ['GB', 'DE', 'US']
store_data = store_data[store_data['country_code'].isin(valid_country_codes)]
store_data['country_code'] = store_data['country_code'].astype('category')

continent_corrections = { 'eeEurope': 'Europe', 'eeAmerica': 'America'}
store_data['continent'] = store_data['continent'].replace(continent_corrections)
rows_to_drop = store_data[store_data['continent'].isin(['NULL', 'QMAVR5H3LD', 'LU3E036ZD9', 
                                           '5586JCLARW', 'GFJQ2AAEQ8', 'SLQBD982C0', 
                                           'XQ953VS0FG', '1WZB1TE1HL'])].index
store_data = store_data.drop(rows_to_drop)


In [ ]:
print(product_df['weight'])

In [ ]:
print(store_data['lat'].unique())


In [ ]:
print(store_data.info())


In [ ]:
duplicated = stores.duplicated(subset=['address'])

stores[duplicated]

In [ ]:
try:
    with open('api_creds.yaml', 'r') as file:
        config = yaml.safe_load(file)
    api_key = config['api']['x-api-key']
except FileNotFoundError:
    print("The YAML configuration file was not found.")
except KeyError as e:
    print(f"A key error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


'J78': Contains a letter.
'30e': Contains a letter, looks like it could be scientific notation but in the context of staff numbers, this is likely not valid.
'80R': Contains a letter.
'A97': Contains a letter.
'3n9': Contains a letter.


In [ ]:
print(store_data['staff_numbers'].unique())

staff_corrections = {'J78' : '78', '30e':'30', '80R':'80', 'A97' :'97', "3n9" : '39'}


In [ ]:
print(df_2['card_provider'].unique())

In [ ]:
print(df_2['expiry_date'].unique())

In [ ]:
print(df_2['card_number'].unique())

In [ ]:
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

In [ ]:
DatabaseConnector.list_db_tables()
orders_table = DataExtractor.read_rds_table('orders_table')


In [ ]:
orders_table

In [ ]:
def clean_orders_data(orders_table):
    clean_orders_table = orders_table.drop(['level_0', 'first_name', 'last_name', 'index', "1"], axis = 1)
    return clean_orders_table

In [ ]:
cleaned_table = clean_orders_data(orders_table)
cleaned_table.info()

In [ ]:
DataExtractor.extract_from_s3('https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json')


In [1]:
import requests
import pandas as pd 

response = requests.get("https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json")

if response.status_code != 200:
    raise Exception(f'Failed to fetch data: {response.status_code}')
content = response.content.decode('utf-8')
events_data = pd.read_json(content)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
events_data.to_csv('events_data.csv')

In [ ]:
events_data

In [ ]:
events_data.info()

In [2]:

# 2. Combine 'year', 'month', and 'day' into a single date string
events_data['combined_date'] = events_data['year'].astype(str) + '-' + \
                                     events_data['month'].astype(str) + '-' + \
                                     events_data['day'].astype(str)

# 3. Convert the combined date string to datetime and drop rows where conversion fails
events_data['date'] = pd.to_datetime(events_data['combined_date'], errors='coerce')
events_data = events_data.dropna(subset=['date'])

# 4. Drop the now redundant 'year', 'month', 'day', and 'combined_date' columns
events_data = events_data.drop(['year', 'month', 'day', 'combined_date'], axis=1)


# Display the cleaned DataFrame



In [6]:

events_data['timestamp'] = pd.to_datetime(events_data['timestamp'], format='%H:%M:%S').dt.time
events_data

events_data['time_period'].unique()


<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 120160
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    120123 non-null  object        
 1   time_period  120123 non-null  object        
 2   date_uuid    120123 non-null  object        
 3   date         120123 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 4.6+ MB


In [7]:
events_data['time_period'].unique()

array(['Evening', 'Morning', 'Midday', 'Late_Hours'], dtype=object)